In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r"C:\Users\govin\Downloads\data.csv.zip")
data.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [3]:
df = data.copy()

In [1]:
df.columns

NameError: name 'df' is not defined

In [5]:
df.dropna(subset=["title", "authors"], inplace=True)  # Remove rows where title or author is missing
df.drop_duplicates(inplace=True)  # Remove duplicate rows


In [6]:
import re

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"[^a-z0-9\s]", "", text)  # Remove special characters
        return text
    return "" 

df["title"] = df["title"].apply(clean_text)
df["authors"] = df["authors"].apply(clean_text)
df["categories"] = df["categories"].apply(clean_text)
df["description"] = df["description"].apply(clean_text)

In [7]:
def isbn10_to_isbn13(isbn10):
    if isinstance(isbn10, str) and len(isbn10) == 10:
        return "978" + isbn10[:-1]  # Add '978' prefix and remove last digit
    return isbn10  # If already ISBN-13, return as is

df["isbn13"] = df["isbn10"].apply(isbn10_to_isbn13)

In [9]:
df.to_csv(r"C:\Users\govin\Downloads\cleaneds_data.csv.zip", index=False)

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

data = pd.read_csv(r"C:\Users\govin\Downloads\data.csv.zip")

# df = data.copy()

df["description"] = df["description"].fillna("")


tfidf = TfidfVectorizer(stop_words="english", max_features=5000)

tfidf_matrix = tfidf.fit_transform(df["description"])

print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

TF-IDF Matrix Shape: (6738, 5000)


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between books
cosine_sim = cosine_similarity(tfidf_matrix)

# Check the shape of the similarity matrix
print("Cosine Similarity Matrix Shape:", cosine_sim.shape)

Cosine Similarity Matrix Shape: (6738, 6738)


In [12]:
def get_similar_books(book_index, top_n=5):
    # Get similarity scores for the given book
    similarity_scores = list(enumerate(cosine_sim[book_index]))
    
    # Sort by highest similarity scores (excluding itself)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    # Get book titles
    similar_books = [df.iloc[i[0]]["title"] for i in similarity_scores]
    
    return similar_books

# Example: Find similar books for the first book (index 0)
print("Similar books:", get_similar_books(0))

Similar books: ['go tell it on the mountain', 'beyond the hundredth meridian', 'vanity fair', 'the languages of pao', 'i can read with me eyes shut']


In [13]:
def get_recommendations(book_title, top_n=5):
    # Convert title to lowercase for case-insensitive search
    book_title = book_title.lower()
    
    # Find the index of the book in the DataFrame
    indices = df[df["title"].str.lower() == book_title].index

    if len(indices) == 0:
        return "Book not found. Please check the title."

    book_index = indices[0]  # Get the first match

    # Get similarity scores for the book
    similarity_scores = list(enumerate(cosine_sim[book_index]))
    
    # Sort books by highest similarity score (excluding itself)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    recommended_books = [df.iloc[i[0]]["title"] for i in similarity_scores]

    return recommended_books

# Example: Get recommendations for a book
book_name = "Harry Potter and the Sorcerer's Stone"  # Change this to any book title in your dataset
print("Top 5 Recommended Books:", get_recommendations("harry potter"))


Top 5 Recommended Books: ['a light in the attic book and cd', 'good grief', 'how to talk to anyone  92 little tricks for big success in relationships', 'the wish giver', 'dirty italian']
